# 영화 추천 시스템

1. Demographic Filtering (인구통계학적 필터링)
2. Content Based Filtering (컨텐츠기반 필터링)
3. Collaborative Filterinf (협업 필터링)

## 1.  Demographic Filtering (인구통계학적 필터링)

In [ ]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')

In [ ]:
df1.head()

In [ ]:
df2.head(3)

In [ ]:
df1.shape, df2.shape

In [ ]:
df1['title'].equals(df2['title'])

In [ ]:
df1.columns

In [ ]:
df1.columns = ['id', 'title', 'cast', 'crew']
df1.columns

In [ ]:
df1[['id', 'cast', 'crew']]

In [ ]:
df2 = df2.merge(df1[['id', 'cast', 'crew']], on='id')
df2.head(3)

In [ ]:
C = df2['vote_average'].mean()        # 전체 영화에 대한 평균 평점
C

In [ ]:
m = df2['vote_count'].quantile(0.9)    # 상위 10%에 해당하는 평가수를 가진 영화의 기준
m

In [ ]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]     
q_movies.shape

In [ ]:
q_movies['vote_count'].sort_values()

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/ (v + m) * R) + (m / (m + v) * C)

In [ ]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)     # q_movies 데이터에서 함수를 통과해서 산출된 값이 'score'의 값으로 들어감
q_movies.head()

In [ ]:
q_movies = q_movies.sort_values('score', ascending=False)    # 내림차순 정렬
q_movies[['title','vote_count', 'vote_average','score']].head(10)

In [ ]:
pop= df2.sort_values('popularity', ascending=False)
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.barh(pop['title'].head(10),pop['popularity'].head(10), align='center',
        color='skyblue')
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Popular Movies")

## 2. Content Based Filtering (컨텐츠 기반 필터링)

#### [줄거리 기반 추천]

In [ ]:
df2['overview'].head(5)

Bag Of Words - BOW 

문장1 : I am a boy

문장2 : I am a girl

I(2), am(2), a(2), boy(1), girl(1)

        
        
        I    am   a   boy    girl
 문장1  1    1    1    1      0   (1,1,1,1,0)
 (I am a boy)
 
 문장2  1    1    1    0      1   (1,1,1,0,1)
 (I am a girl)
 
 피처 벡터화.
 
 
 문서 100개
 모든 문서에서 나온 단어 10,000 개
 100 * 10,000 = 100만
 
        단어1, 단어2, 단어3, 단어4, .... 단어 10000
문서1    1       1       3    0    
문서2
문서3
..
문서100

1. TfidfVectorizer (TF-IDF 기반의 벡터화)
2. CountVectorizer


### 1. tf_idf(Term Frequency-Inverse Document Frequency)
- 여러 문서로 이루어진 문서군이 있을 때  Document Term Matrix(DTM) 내 단어마다 중요도를 고려하여 가중치를 주는 통계적인 단어 표현방법
- 문서 간 유사도 검사, 중요 문서 위주의 검색 결과 제공 시스템, 문서 빅데이터 내 핵심어 추출 등에 활용
- TF-IDF 값이 클수록 해당 단어의 중요도가 높음
- 단어 출현빈도(TF)와 역 문서빈도(IDF) 를 곱하여 단어마다 가중치 부여

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')       # The, a 같은 영어에서 의미없는 문자들을 제거

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
ENGLISH_STOP_WORDS

In [ ]:
df2['overview'].isnull().values.any()

In [ ]:
df2['overview'] = df2['overview'].fillna('')

In [ ]:
tfidf_matrix = tfidf.fit_transform(df2['overview'])    # 피쳐 백터화
tfidf_matrix.shape                                     # 4803개의 문서가 20978개의 단어로 이루져 있음

In [ ]:
tfidf_matrix                               # 125840개의 0이 아닌 데이터가 존재

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

| | 문장1 | 문장2 | 문장3 |
|---|---|---|---|
|문장1|1|0.3|0.8|
|문장2|0.3|1|0.5|
|문장3|0.8|0.5|1|

In [ ]:
cosine_sim.shape

In [ ]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()     # 영화 제목을 통해서 그 영화가 전체 데이터중에 몇번째에 해당하는지 인덱스 출력
indices                                                                  # drop_duplicates:중복제거 , Series 1차원 배열

In [ ]:
indices['The Dark Knight Rises']

In [ ]:
df2.iloc[[3]]

In [ ]:
# 영화의 제목을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 영화 목록 반환

def get_recommendations(title, cosine_sim=cosine_sim):
        # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기
        idx = indices[title]

        # 코사인 유사도 매트릭스 (cosine_sim)에서 idx에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 코사인 유사도 기준으로 내림차순 정렬
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 자기 자신을 제외한 10개의 추천 영화를 슬라이싱
        sim_scores= sim_scores[1:11]  

        # 추천 영화 목록 10개의 인덱스 정보 추출
        movie_indices = [i[0] for i in sim_scores]

        # 인덱스 정보를 통해 영화 정보 추출
        return df2['title'].iloc[movie_indices]

In [ ]:
test_idx = indices['The Dark Knight Rises']    # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index 값을 얻기
test_idx

In [ ]:
cosine_sim[3]

In [ ]:
list(enumerate(cosine_sim[3]))      # 코사인 유사도 매트릭스 (cosine_sim)에서 idx에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
                                    # 각 인덱스와 각 인덱스별 코사인 유사도

In [ ]:
test_sim_scores = list(enumerate(cosine_sim[3]))   # 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스 값을 포함하는 enumerate 객체를 리턴

In [ ]:
test_sim_scores = sorted(test_sim_scores, key=lambda x: x[1], reverse=True)    # 코사인 유사도 기준으로 내림차순 정렬
test_sim_scores[1:11]   # 자기 자신을 제외한 10 개의 추천 영화를 슬라이싱

In [ ]:
def get_second(x):           # 리스트에서 두번째 값을 얻어오는 함수
    return x[1]

lst = ['인덱스','유사도']
print(get_second(lst))

In [ ]:
(lambda x: x[1])(lst)         

In [ ]:
# 추천 영화 목록 10개의 인덱스 정보 추출
test_movie_indices = [i[0] for i in test_sim_scores[1:11]]
test_movie_indices

In [ ]:
# 인덱스 정보를 통해 영화 정보 추출
df2['title'].iloc[test_movie_indices]

In [ ]:
df2['title'][:20]

In [ ]:
get_recommendations('Avengers: Age of Ultron')     # ()안에 영화를 기준으로 코사인 유사도가 가장 높은 10개의 영화 추천

In [ ]:
get_recommendations('The Avengers')

### [다양한 요소 기반 추천(장르, 감독, 키워드)]

In [ ]:
df2.head(3)

In [ ]:
df2.loc[0,'genres']

In [ ]:
s1  = [{"id": 28, "name": "Action"}]
s2  = '[{"id": 28, "name": "Action"}]'

In [ ]:
type(s1), type(s2)

In [ ]:
from ast import literal_eval
s2 = literal_eval(s2)
s2, type(s2)

In [ ]:
print(s1)
print(s2)

In [ ]:
features = ['cast', 'crew', 'keywords', 'genres'] 
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)     # features안에 있는 컬럼을 가진 데이터들의 type을 문자형에서 자료형으로 변경

In [ ]:
df2.loc[0, 'crew']

In [ ]:
# 감독 정보를 추출
def  get_directer(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
df2['director'] = df2['crew'].apply(get_directer)
df2['director']

In [ ]:
df2[df2['director'].isnull()]

In [ ]:
df2.loc[0, 'cast']

In [ ]:
df2.loc[0, 'genres']

In [ ]:
df2.loc[0, 'keywords']

In [ ]:
# 처음 3개의 데이터 중에서 name에 해당하는 value만 추출
def get_list(x):
    if isinstance(x, list):          # x가 리스트 인스턴스인지 확인
        names = [i['name'] for i in x]    
        if len(names) > 3:
            names = names[:3]
        return names
    return []


In [ ]:
features = ['cast', 'keywords','genres']
for feature in features :
    df2[feature] = df2[feature].apply(get_list)

In [ ]:
df2[['title','cast','director','keywords','genres']].head(3)

In [ ]:
def clean_data(x): 
    if isinstance(x, list):
        return [str.lower(i.replace(' ','')) for i in x]             # 데이터를 소문자로 변경/ 빈 값 채우기
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ',''))
        else:
            return''

In [ ]:
features = ['cast', 'keywords','director','genres']
for feature in features :
    df2[feature] = df2[feature].apply(clean_data)

In [ ]:
df2[['title','cast','director','keywords','genres']].head(3)

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)
df2['soup']

### 2. CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])
count_matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2

In [ ]:
indices['Avatar']

In [ ]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])
indices

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

In [ ]:
get_recommendations('Up', cosine_sim2)

In [ ]:
import pickle

In [ ]:
df2.head(3)

In [ ]:
movies = df2[['id','title']].copy()
movies.head(5)

In [ ]:
pickle.dump(movies, open('movies.pickle','wb'))       # movies에 있는 정보를 저장

In [ ]:
pickle.dump(cosine_sim2, open('cosine_sim.pickle','wb'))

In [ ]:
with open ('movies.pickle','rb') as movies_file:     # 파일 불러오기
    print(pickle.load(movies_file))